# Welcome to assignment 1: Exhaustive Search & Backtracking

## Build a Sudoku Solver

In this week's programming assignment, you will be working on a sudoku solver. Throughout the assignment, you will step by step build this sudoku solver. You do not need to copy the code, it is enough to work in the cell under each assignment. Note that there are separate cells provided where you can (and should) test your code. During the assignment, you will (through customMagics) obtain a Python file (.py) which you should run against a set of unittests. Please avoid writing any unnecessary code in cells containing the `%%execwritefile` command. Doing this could alter the file `.py` and make it syntactically incorrect or interfere with the unittests. To prevent this stick to the following rules:'
 - ***Do not remove cells that start with ``%%execwritefile`` and do not remove that line.***
 - If a cell contains a `%%execwritefile` command at the top and a class definition you need to complete the given methods and adding helper methods is allowed, but do **not** add new functions or Python script to the cells (like global variables).
 - If a cell contains a `%%execwritefile` command at the top and **not** a class definition you must complete the given functions and you are free to add helper functions, new classes, and Python script that contains for example global variables. Note, that the use of global variables is almost always wrong except for a few use cases such as RNG for the numpy random generator methods.
 - If a cell does **not** contain a `%%execwritefile` command you can plot things, print variables, and write test cases. Here, you are free to do whatever you want.
 - If a cell does **not** contain a `%%execwritefile` command it should not contain functional code that is needed to run other functions or classes. The reason is that it is not copied to the `.py`. So, it can not be used during the unittesting.

You do not need to look at the customMagic.py nor do more than glimpse at the test file, your assignment is contained in this workbook unless specified differently in this notebook's instructions. 

This assignment is given as a Jupyter notebook, you might run this in your browser by starting a notebook server or through VScode (PyCharm only allows for read-only mode for jupyter notebooks, unless you have Pycharm-pro which is a paid version or you need a student license). The simplest way to [install jupyter](https://docs.jupyter.org/en/latest/install/notebook-classic.html) notebooks is by installing [Anaconda](https://docs.anaconda.com/free/anaconda/install/), a custom python distribution and packagemanager used for Data Science. If you do not want to install all of Anaconda you can also install jupyter via pip: ``pip3 install jupyter`` (`pip3` or `pip` will work depending on how you installed python3). You can find a tutorial for running the notebook [here](https://jupyter-notebook.readthedocs.io/en/latest/notebook.html). With VScode the IDE should guide you in installing the correct packages. 

***Hint: Jupyter Notebooks saves variables between runs. If you get unexpected results try restarting the kernel, this deletes any saved variables.*** 

### Some Additional Rules:

 - You are not allowed to change the given code. This includes attribute names, method names, arguments, etc.
 - You are not allowed to import other modules than the one provided.
 - You are allowed to add functions yourself if you feel that that makes it easier. Note, however, that points are deducted if we think that they are unnecessary. Make sure to document these consistently.
 - Read the written documentation about the functions you need to implement, they tell you what to do. Most of these functions require less than 10 lines of code.
 - A few sudoku grids are given as .txt files. Use these to test your code. You are allowed to add more test files.
 - In the end, you can run unittests as an extra check that your code works. You are free to add additional unittests.
 - If your program does not succeed on all unittests that are provided, it is likely that there is still a problem in your code. Make sure that all unittests succeed, before submitting the code.
 - Also keep in mind that all unit tests should be able to run within a matter of seconds on any computer.

Please fill in your student name down below 

In [ ]:
# FILL IN YOU STUDENT NUMBER
student = 1000000

# Set this to false if you want the default screen width.
WIDE_SCREEN = True

In [ ]:
from custommagics import CustomMagics

if WIDE_SCREEN:
    import notebook
    from IPython.display import display, HTML

    if int(notebook.__version__.split(".")[0]) >= 7:    
        display(HTML(
            '<style>'
                '.jp-Notebook { padding-left: 1% !important; padding-right: 1% !important; width:100% !important; } '
            '</style>'
        ))
    else:
        display(HTML("<style>.container { width:98% !important; }</style>"))

get_ipython().register_magics(CustomMagics)

In [ ]:
%%execwritefile assignment1_{student}_notebook.py 0 

# DO NOT CHANGE THIS CELL.
# THESE ARE THE ONLY IMPORTS YOU ARE ALLOWED TO USE:

import numpy as np
import copy

RNG = np.random.default_rng()

## CSPs 

Many real-world problems can be formulated as [Constraint Satisfaction Problems (CSPs)](https://en.wikipedia.org/wiki/Constraint_satisfaction_problem#:~:text=Constraint%20satisfaction%20problems%20(CSPs)%20are,solved%20by%20constraint%20satisfaction%20methods.), where
we have a set of constraints that a valid solution must satisfy. Examples of such CSPs include
planning and scheduling problems, but also the n-Queens problem, crossword puzzles, and
sudokus. In this assignment, you will program a recursive exhaustive search algorithm
that aims to find solutions for the sudoku described below.

### Sudoku Description

A sudoku is a puzzle where you have a $n \times n$ 2D grid where each row, column, and box needs to contain the numbers 1 to n. A box is a $\sqrt{n} \times \sqrt{n}$ grid inside the nxn grid. For more information about sudokus read the [wiki page](https://en.wikipedia.org/wiki/Sudoku). Make sure you understand the problem before continuing reading. Below, you can find an image of a traditional 9x9 sudoku with the following naming conventions, the rows are red, the columns blue and the boxes are green.

![sudoku.png](sudoku.png)

## 1.0 Initialize The Sudoku Class

In the cell below, you start implementing the sudoku solver. To start, study how this class works, and ensure you thoroughly study the docstrings to get a clear idea of how the class ought to be implemented. 

Your task is to complete the `__repr__` magic method. This is for easier bug fixing. For example, if you want to print the sudoku to see if your code works it does not help if you get a memory address. 
There is no format that you need to adhere to nor is this mandatory.

In [ ]:
%%execwritefile assignment1_{student}_notebook.py 10 -a -s

class Sudoku():
    """
    This class creates sudoku objects which can be used to solve sudokus. 
    A sudoku object can be any size grid, as long as the square root of the size is a whole integer.
    To indicate that a cell in the sudoku grid is empty we use a zero.
    A sudoku object is initialized with an empty grid of a certain size.

    Attributes:
        :param self.grid: The sudoku grid containing all the digits.
        :type self.grid: np.ndarray[(Any, Any), int]  # The first type hint is the shape, and the second one is the dtype. 
        :param self.size: The width/height of the sudoku grid.
        :type self.size: int
    """
    def __init__(self, size=9):
        self.grid = np.zeros((size, size))
        self.size = size
        
    def __repr__(self):
        """
        This returns a representation of a Sudoku object.

        :return: A string representing the Sudoku object.
        :rtype: str
        """
        # Change this to anything you like, such that you can easily print a Sudoku object.
        return super(Sudoku, self).__repr__() 

### Test your code

In the cell below you can test your code from the cell above (ensure you run the cell beforehand) to validate if `__repr__` works as you intended.
To do this create a `Sudoku` object and print it.

In [ ]:
Sudoku()  # jupyter notebook cells always print the return of the last line of code. So `a=3` will not print anything but `a` will print the variable a.

## 1.1 Make The Sudoku Class Testable

While a lot of methods that we will implement can be tested using an empty grid, 
It is helpful to build something that can test if your code is performing the way it should by partially filling in the sudoku. 
To achieve this we need two things. A function that can read in some text files with partial sudokus and a method in the class sudoku that can set this initial state.

This method will be part of the .py file that you hand in. However, this read function will not be part of your file. The reason for that is that we want you to learn to test your own code and not rely on given unittests. 

In [ ]:
%%execwritefile assignment1_{student}_notebook.py 11 -a -s -c

class Sudoku(Sudoku):
    def set_grid(self, grid):
        """
        This method sets a new grid. This also can change the size of the sudoku.

        :param grid: A 2D numpy array that contains the digits for the grid.
        :type grid: ndarray[(Any, Any), int]
        """
        raise NotImplementedError("Please complete this method")

In [ ]:
def sudoku_file_to_object(filename):
    """
    Open the text file and convert the raw string into a 2D sudoku grid.

    :return: A Sudoku object containing the grid that is in the file.
    :rtype: Sudoku
    """
    raise NotImplementedError("Please complete this method")

### Test your code

In the cell below you can test your code from the cell above (ensure you run the cell beforehand). 
Test if your `sudoku_file_to_object` function works correctly. Don't forget to also test if the size is correct.
Additionally, you can test if `set_grid` works separately instead of implicitly.

In contrast to the exercises, here we will no longer give the almost full scripts to test your code. At most, we will give you one line of code to remind you to test your previous code.
It is important that you write small scripts similar to what you have seen in the exercise problems, such that you solve all bugs when you create them. 
If you do not test your code before running the unittests you have a high risk of accumulating a nasty combination of bugs which makes it hard to bugfix.

In [ ]:
sudoku_file_to_object("small_test.txt")

## 1.2 Helper Methods

Helper functions can make your code more structured and split problems into their separate subproblems. This also increase the readability of your code. Another benefit is that it also improves the testability of your class. This improvement can be mainly found in the fact that you can now test the helper function separately from other methods. For example, if your `get_box` does not retrieve the right slice of the grid then you will catch the mistake before using it in other methods. Again as said before, solving one bug is easier than a whole nest of bugs that all interact with each other.

Below, you will implement `get_row`, `get_col`, and `get_box`.

In [ ]:
%%execwritefile assignment1_{student}_notebook.py 12 -a -s -c

class Sudoku(Sudoku):
    def get_row(self, row_id):
        """
        This method returns the row with index row_id.

        :param row_id: The index of the row.
        :type row_id: int
        :return: A row of the sudoku.
        :rtype: np.ndarray[(Any,), int]
        """
        raise NotImplementedError("Please complete this method")

    def get_col(self, col_id):
        """
        This method returns the column with index col_id.

        :param col_id: The index of the column.
        :type col_id: int
        :return: A row of the sudoku.
        :rtype: np.ndarray[(Any,), int]
        """
        raise NotImplementedError("Please complete this method")

    def get_box_index(self, row, col):
        """
        This returns the box index of a cell given the row and column index.
        
        :param col: The column index.
        :type col: int
        :param row: The row index.
        :type row: int
        :return: This returns the box index of a cell.
        :rtype: int
        """
        raise NotImplementedError("Please complete this method")

    def get_box(self, box_id):
        """
        This method returns the "box_id" box.

        :param box_id: The index of the sudoku box.
        :type box_id: int
        :return: A box of the sudoku.
        :rtype: np.ndarray[(Any, Any), int]
        """
        raise NotImplementedError("Please complete this method")

### Test your code

In the cell below you can test your code from the cell above (ensure you run the cell beforehand).
Test if your `get_row`, `get_col`, and `get_box`. methods works correctly. 

Just as a reminder: \
In contrast to the exercises, here we will no longer give the almost full scripts to test your code. At most, we will give you one line of code to remind you to test your previous code.
It is important that you write small scripts similar to what you have seen in the exercise problems, such that you solve all bugs when you create them. 
If you do not test your code before running the unittests you have a high risk of accumulating a nasty combination of bugs which makes it hard to bugfix.

In [ ]:
sudoku_file_to_object("small_test.txt")

## 1.3 Check Methods

It is a good habit to split up one method that does multiple things into several methods that do one thing. Here, our end goal is to make a sudoku solver. While it is possible to have one method `solve` which does everything, it is much more readable, testable, and overall better practice to split up such large methods into smaller methods. 

In the cell below, we will implement various check methods that return if a set of numbers is unique (`is_set_correct`), if a cell is correct (`check_cell`), and if the (partial) sudoku is correct (`check_soduko`). Note, that not every method needs to be used right away, some are used later in the notebook.

In [ ]:
%%execwritefile assignment1_{student}_notebook.py 13 -a -s -c

class Sudoku(Sudoku):
    @staticmethod
    def is_set_correct(numbers):
        """
        This method checks if a set (row, column, or box) is correct according to the rules of a sudoku.
        In other words, this method checks if a set of numbers contains duplicate values between 1 and the size of the sudoku.
        Note, that multiple empty cells are not considered duplicates.

        :param numbers: The numbers of a sudoku's row, column, or box.
        :type numbers: np.ndarray[(Any, Any), int] or np.ndarray[(Any, ), int]
        :return: This method returns if the set is correct or not.
        :rtype: Boolean
        """
        raise NotImplementedError("Please complete this method")

    def check_cell(self, row, col):
        """
        This method checks if the cell, denoted by row and column, is correct according to the rules of sudoku.
        
        :param col: The column index that is tested.
        :type col: int
        :param row: The row index that is tested.
        :type row: int
        :return: This method returns if the cell, denoted by row and column, is correct compared to the rest of the grid.
        :rtype: boolean
        """
        raise NotImplementedError("Please complete this method")

    def check_sudoku(self):
        """
        This method checks, for all rows, columns, and boxes, if they are correct according to the rules of a sudoku.
        In other words, this method checks, for all rows, columns, and boxes, if a set of numbers contains duplicate values between 1 and the size of the sudoku.
        Note, that multiple empty cells are not considered duplicates.

        Hint: It is not needed to check if every cell is correct to check if a complete sudoku is correct.

        :return: This method returns if the (partial) Sudoku is correct.
        :rtype: Boolean
        """
        raise NotImplementedError("Please complete this method")

### Test your code

In the cell below you can test your code from the cell above (ensure you run the cell beforehand). 
Test if all three methods (`is_set_correct`, `check_cell`, `check_soduko`) are correctly implemented before continuing to the next set of methods.

In [ ]:
sudoku_file_to_object("small_test.txt")

## 1.4 Exhaustive Search

Now, that we have all the helper methods, we can start implementing the exhaustive search to solve a sudoku. Below, you will find various methods, please keep to the following rules including all the rules stated at the beginning of this file:
 - `solve` is already implemented, do not change this method.
 - You have to implement `step` which is the recursive part of the exhaustive search.
 - `next_step` and `clean_up` are not mandatory to use, these are some helper methods to structure your recursion. If you do not use them, leave them as is.
 - You are allowed to add extra **methods** if you feel that it helps you. Note, however, that points are deducted if we think that they are unnecessary. Make sure to document these consistently.
 - Just as a reminder, you are not allowed to add **functions** in the cell below or anything else outside the class sudoku.
 - Do not change the arguments of the methods, for now, you can ignore the default argument `backtracking`.

In [ ]:
%%execwritefile assignment1_{student}_notebook.py 14 -a -s -c

class Sudoku(Sudoku):
    def step(self, row=0, col=0, backtracking=False):
        """
        This is a recursive method that completes one step in the exhaustive search algorithm.
        A step should contain at least, filling in one number in the sudoku and calling "next_step" to go to the next step.
        If the current number for this step does not give a correct solution another number should be tried 
        and if no numbers work the previous step should be adjusted.
        
        Hint 1: Numbers, that are already filled in should not be overwritten.
        Hint 2: Think about a base case.
    
        :param col: The current column index.
        :type col: int
        :param row: The current row index.
        :type row: int
        :param backtracking: This determines if backtracking is used. For now, this can be ignored. It defaults to False.
        :type backtracking: boolean, optional
        :return: This method returns if a correct solution can be found using this step.
        :rtype: boolean
        """
        raise NotImplementedError("Please complete this method")

    def next_step(self, row, col):
        """
        This method calculates the next step in the recursive exhaustive search algorithm.
        This method should only determine which cell should be filled in next.
        
        :param col: The current column index.
        :type col: int
        :param row: The current row index.
        :type row: int
        :param backtracking: This determines if backtracking is used. For now, this can be ignored. It defaults to False.
        :type backtracking: boolean, optional
        :return: This method returns if a correct solution can be found using this next step.
        :rtype: boolean
        """
        raise NotImplementedError("Please complete this method")
    
    def clean_up(self, row, col):
        """
        This method cleans up the current cell if no solution can be found for this cell.
        
        :param col: The current column index.
        :type col: int
        :param row: The current row index.
        :type row: int
        :return: This method returns if a correct solution can be found using this next step.
        :rtype: boolean
        """
        raise NotImplementedError("Please complete this method")
    
    def solve(self, backtracking=False):
        """
        Solve the sudoku using recursive exhaustive search.
        This is done by calling the "step" method, which does one recursive step.
        This can be visualized as a process tree, where "step" completes the functionality of of node.
        
        This method is already implemented and you do not have to do anything here.

        :param backtracking: This determines if backtracking is used. For now, this can be ignored. It defaults to False.
        :type backtracking: boolean, optional
        :return: This method returns if a correct solution for the whole sudoku was found.
        :rtype: boolean
        """
        return self.step(backtracking=backtracking)

### Test your code

In the cell below you can test your code from the cell above (ensure you run the cell beforehand). 

***Important, think about how big the state space is for a Sudoku of $n \times n$ if completely empty! Would it be possible to run an exhaustive search on an empty $n \times n$ grid and get a solution in real-time? In other words, think about the complexity of your algorithm and the size of the problem, thus if it will run in a few seconds.***

Hint: While a sudoku of size 2 or 3 is not an official size because the boxes are not well-defined, you can use it to test your algorithm for a problem set with a reduced size.

In [ ]:
sudoku_file_to_object("small_test.txt").solve()

## 1.5 Backtracking

Below, we will implement backtracking instead of an exhaustive search. This will help to find solutions for larger sudokus. Note, that this will be covered in lecture 5. In short the main difference is that backtracking does not wait till you find an end state to check if the solution is correct but also checks each partial solution (state) and if it is incorrect discard the state-space and all states that lead from there.

***Important, in the cell below you will overwrite the methods implemented in the cell above. Therefore, even if the methods are the same you need to implement them again (copy-paste), otherwise your `.py` file will not work! Also, because you overwrite the methods the code below should work for both exhaustive search and backtracking.***

Hint: The code for backtracking should almost be a direct copy-paste from the exhaustive search implemented above with approximately 2 extra lines of extra code.

In [ ]:
%%execwritefile assignment1_{student}_notebook.py 14 -a -s -c

class Sudoku(Sudoku):       
    def step(self, row=0, col=0, backtracking=False):
        """
        This is a recursive method that completes one step in the exhaustive search algorithm.
        A step should contain at least, filling in one number in the sudoku and calling "next_step" to go to the next step.
        If the current number for this step does not give a correct solution another number should be tried 
        and if no numbers work the previous step should be adjusted.

        This method should work for both backtracking and exhaustive search.
        
        Hint 1: Numbers, that are already filled in should not be overwritten.
        Hint 2: Think about a base case.
        Hint 3: The step method from the previous jupyter notebook cell can be copy-paste here and adjusted for backtracking.
    
        :param col: The current column index.
        :type col: int
        :param row: The current row index.
        :type row: int
        :param backtracking: This determines if backtracking is used, defaults to False.
        :type backtracking: boolean, optional
        :return: This method returns if a correct solution can be found using this step.
        :rtype: boolean
        """
        raise NotImplementedError("Please complete this method")

    def next_step(self, row, col, backtracking):
        """
        This method calculates the next step in the recursive exhaustive search algorithm.
        This method should only determine which cell should be filled in next.

        This method should work for both backtracking and exhaustive search.
        
        :param col: The current column index.
        :type col: int
        :param row: The current row index.
        :type row: int
        :param backtracking: This determines if backtracking is used, defaults to False.
        :type backtracking: boolean, optional
        :return: This method returns if a correct solution can be found using this next step.
        :rtype: boolean
        """
        raise NotImplementedError("Please complete this method")
    
    def clean_up(self, row, col):
        """
        This method cleans up the current cell if no solution can be found for this cell.

        This method should work for both backtracking and exhaustive search.
        
        :param col: The current column index.
        :type col: int
        :param row: The current row index.
        :type row: int
        :return: This method returns if a correct solution can be found using this next step.
        :rtype: boolean
        """
        raise NotImplementedError("Please complete this method")
    
    def solve(self, backtracking=False):
        """
        Solve the sudoku using recursive exhaustive search or backtracking.
        This is done by calling the "step" method, which does one recursive step.
        This can be visualized as a process tree, where "step" completes the functionality of of node.
        
        This method is already implemented and you do not have to do anything here.

        :param backtracking: This determines if backtracking is used, defaults to False.
        :type backtracking: boolean, optional
        :return: This method returns if a correct solution for the whole sudoku was found.
        :rtype: boolean
        """
        return self.step(backtracking=backtracking)

### Test your code

In the cell below you can test your code from the cell above (ensure you run the cell beforehand). 

Note, that solve now has two options exhaustive search with or without backtracking.

In [ ]:
sudoku_file_to_object("small_test.txt").solve(True)

# UNITTESTS

During this assignment, we copied all your code to the following **.py** file **"assignment1_{student}_notebook.py"**. You also tested your code along the way. However, it is possible that there are still a few errors. Therefore, it is good to run some unittest when you complete all coding. This gives you an extra chance to spot mistakes. Here, we added some unittest for you to use. Note, that they are not ***complete*** and that they are merely an indication if you are above or below a 6 (still no guarantee). 

From this point onwards we strongly advise renaming the **"assignment1_{student}_notebook.py"** file to the correct file name that you need to hand in **"assignment1_{student}.py"**. Now, you can adjust the **"assignment1_{student}.py"** file without the risk of overwriting it when you run the notebook again. This also enables the possibility to run the unittests. Note, that from now on you are done programming in the notebook and you need to adjust the **.py** file to fix bugs. To run the unittests go to the **"unit_test.py"** file and run the file in either PyCharm, VSCode, or a terminal. You can run it in a terminal using the following command: `python -m unittest --verbose unit_test.py`. `--verbose` is optional but gives you more details about which tests fail and which succeed.

You are allowed to add your own unittests.

***Do not forget to write Your Report! Instruction can be found below***

# Report

Write a report in LATEX(at most 3 pages) using the provided template (see Brightspace), addressing the following points/research questions:
 - Introduction: describe the problem. Describe a state and action explicitly in the context of this problem.
 - Explain the difference between exhaustive search and backtracking. What makes the implemented approach exhaustive search? How would you change to make it do backtracking instead?
 - Is there a difference in complexity between the implemented exhaustive search and backtracking? What are the complexities of both algorithms?
 - Draw the search tree of a small example that an exhaustive search traverses. Additionally, draw the same search tree, but then for backtracking. What can we conclude? Is backtracking optimal (guaranteed to find a solution if it exists)? You are allowed to draw this on paper and scan the result.
 - How could you make this algorithm even "smarter"?
 - How big is the state space for an $n \times n$ sudoku and would it be possible to run an exhaustive search on an empty $9 \times 9$ grid and get a solution in real-time (time you are willing to wait for the answer)?
 - Summary and Discussion. What was the goal of the assignment? What have you done and observed? (think about backtracking vs exhaustive search, optimality, running time, smart approaches vs exhaustive search and backtracking). Do not write about your personal experience and stories. Keep it scientific and simply summarize the report, making observations about the algorithms.

## Work distribution

At the end of the report, include a distribution of the work: who did what? By default, we
give both group members the same grade, but in some extreme cases, we will adjust the grades according to the workload. The work distribution does not count towards the page limit.

# Submission

Submit your assignment through Brightspace by submitting the following files:
 - report.pdf (the report)
 - assignment1_{student}.py (your solution code)
 - assignment1.ipynb (backup if something goes wrong)
   
The deadline for this assignment is the 8 of April 2023, 23:59 CET.